In [ ]:
import os
import flash
import mlflow
import torch
from flash.core.data.utils import download_data
from flash.text import TextClassificationData, TextClassifier

In [ ]:
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://100.65.79.57:9000/"

In [ ]:
download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", "./data/")
datamodule = TextClassificationData.from_csv(
    "review",
    "sentiment",
    train_file="data/imdb/train.csv",
    val_file="data/imdb/valid.csv",
    test_file="data/imdb/test.csv",
    batch_size=4,
)

In [ ]:
classifier_model = TextClassifier(backbone="prajjwal1/bert-tiny", labels=datamodule.labels, num_classes=datamodule.num_classes)
trainer = flash.Trainer(max_epochs=1, accelerator="mps", devices=1)

In [ ]:
EXPERIMENT_NAME = "dl_model_chapter03"
mlflow.set_tracking_uri('http://100.65.79.57/')
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

In [ ]:
mlflow.pytorch.autolog()

with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="chapter03") as dl_model_tracking_run:
    trainer.finetune(classifier_model, datamodule=datamodule, strategy="freeze")
    #trainer.test()

In [ ]:
run_id = dl_model_tracking_run.info.run_id
print("run_id: {}; lifecycle_stage: {}".format(run_id,
    mlflow.get_run(run_id).info.lifecycle_stage))

In [ ]:
# use the run_id to construct a logged_model URI. An example is shown here:
# logged_model = 'runs:/37a3fe9b6faf41d89001eca13ad6ca47/model'
logged_model = f'runs:/{run_id}/model'


# Load model as a pytorch model, not as the pyfunc model
model = mlflow.pytorch.load_model(logged_model)

#model.trainer.predict({'What a piece of disappointing news'})


In [ ]:
trainer = flash.Trainer(max_epochs=1, accelerator="mps", devices=1)
datamodule = TextClassificationData.from_lists(
    predict_data=[
        "Turgid dialogue, feeble characterization - Harvey Keitel a judge?.",
        "The worst movie in the history of cinema.",
        "This guy has done a great job with this movie!",
    ],
    batch_size=4,
)
predictions = trainer.predict(model, datamodule=datamodule, output="labels")
print(predictions)

In [ ]:
# register the model
model_registry_version = mlflow.register_model(logged_model, 'nlp_dl_model')
print(f'Model Name: {model_registry_version.name}')
print(f'Model Version: {model_registry_version.version}')